## Bootstrap for Estimating Standard Errors of Logistic Regression Coefficients

In [29]:
import os
os.chdir("/Users/darrenz/Downloads")

In [30]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.linear_model import Lasso, LassoCV

%matplotlib inline
sns.set_style("darkgrid")

In [31]:
df = pd.read_csv('Default.csv')
df

,Unnamed: 0,default,student,balance,income
0,1,No,No,729.526495,44361.62507
1,2,No,Yes,817.180407,12106.13470
2,3,No,No,1073.549164,31767.13895
3,4,No,No,529.250605,35704.49394
4,5,No,No,785.655883,38463.49588
...,...,...,...,...,...
9995,9996,No,No,711.555020,52992.37891
9996,9997,No,No,757.962918,19660.72177
9997,9998,No,No,845.411989,58636.15698
9998,9999,No,No,1569.009053,36669.11236


## a)

In [32]:
formula = 'default ~ income + balance'
default_glm = smf.glm(formula, data=df, family=sm.families.Binomial()).fit()
print(default_glm.bse)
default_glm.summary()

Intercept    0.434772
income       0.000005
balance      0.000227
dtype: float64


<class 'statsmodels.iolib.summary.Summary'>
"""
                        Generalized Linear Model Regression Results                        
===========================================================================================
Dep. Variable:     ['default[No]', 'default[Yes]']   No. Observations:                10000
Model:                                         GLM   Df Residuals:                     9997
Model Family:                             Binomial   Df Model:                            2
Link Function:                               logit   Scale:                          1.0000
Method:                                       IRLS   Log-Likelihood:                -789.48
Date:                             Thu, 14 Apr 2022   Deviance:                       1579.0
Time:                                     17:18:41   Pearson chi2:                 6.95e+03
No. Iterations:                                  9                                         
Covariance Type:                         nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     11.5405      0.435     26.544      0.000      10.688      12.393
income     -2.081e-05   4.99e-06     -4.174      0.000   -3.06e-05    -1.1e-05
balance       -0.0056      0.000    -24.835      0.000      -0.006      -0.005
==============================================================================
"""

From the summary of the logistic regression, we can see the coefficient of `income` is -2.081e-05 with an estimated standard error of 4.99e-06. The coefficient of `balance` is -0.0056 with an estimated standard error of 0.000227. 

## b)

In [52]:
def boot_fn(df, index): 
    formula = 'default ~ income + balance'
    df_select = df.loc[index]
    df_glm = smf.glm(formula, data=df_select, family=sm.families.Binomial()).fit()
    return df_glm.params[1:3]

In [53]:
index_count = random.randint(1,10000)

df_sample = df.sample(n = index_count, replace = True) 
df_index = np.array(df_sample.index)

boot_fn(df, df_index) #test the function above

income    -0.000020
balance   -0.005695
dtype: float64

## c)

In [35]:
np.random.seed(123) #set the seed for consistent results

In [36]:
from sklearn.utils import resample

def boot(data, fn, R): #define a function that works as the boot() function in Rstudio
    df_co = pd.DataFrame()
    list1 = []
    list2 = []
    list_se = []
    
    for num in range(R):
        df_resample = resample(data, replace = True) #we use the same size of the dataset for each bootstrap sample
        df_index = np.array(df_resample.index)
        list1.append(list(boot_fn(data, df_index))[0])
        list2.append(list(boot_fn(data, df_index))[1])
    df_co['income_coef'] = list1
    df_co['balance_coef'] = list2
    
    list_se.append(df_co['income_coef'].std())
    list_se.append(df_co['balance_coef'].std())
    
    return list_se        

In [37]:
boot(df, boot_fn, 2000) #choose a number of replicates of 3000 to test the function

[4.787838502790651e-06, 0.00022592076250269424]

## d)

By comparison, we can see that the standard error obtained by bootstrapping is very much similar to the standard error of the logistic regression model.  

## Lasso, Ridge, and Least Squares

## a)

In [10]:
df1 = pd.read_csv('College.csv')
df1.rename(columns={'Unnamed: 0': 'CollegeName'}, inplace = True)
df1.set_index('CollegeName', inplace = True) #Data preprocessing
df1

,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
CollegeName,,,,,,,,,,,,,,,,,,
Abilene Christian University,Yes,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
Adelphi University,Yes,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
Adrian College,Yes,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54
Agnes Scott College,Yes,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59
Alaska Pacific University,Yes,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Worcester State College,No,2197,1515,543,4,26,3089,2029,6797,3900,500,1200,60,60,21.0,14,4469,40
Xavier University,Yes,1959,1805,695,24,47,2849,1107,11520,4960,600,1250,73,75,13.3,31,9189,83
Xavier University of Louisiana,Yes,2097,1915,695,34,61,2793,166,6900,4200,617,781,67,75,14.4,20,8323,49


In [11]:
df1 = pd.get_dummies(df1, columns=['Private'], drop_first=True) #binary coding for the variable 'private'
df1

,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate,Private_Yes
CollegeName,,,,,,,,,,,,,,,,,,
Abilene Christian University,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60,1
Adelphi University,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56,1
Adrian College,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54,1
Agnes Scott College,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59,1
Alaska Pacific University,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Worcester State College,2197,1515,543,4,26,3089,2029,6797,3900,500,1200,60,60,21.0,14,4469,40,0
Xavier University,1959,1805,695,24,47,2849,1107,11520,4960,600,1250,73,75,13.3,31,9189,83,1
Xavier University of Louisiana,2097,1915,695,34,61,2793,166,6900,4200,617,781,67,75,14.4,20,8323,49,1


In [12]:
cols = ['Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F.Undergrad',
 'P.Undergrad',
 'Outstate',
 'Room.Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S.F.Ratio',
 'perc.alumni',
 'Expend',
 'Grad.Rate',
 'Private_Yes',
 'Apps']

df1 = df1[cols] #reorder the column names of data frame
df1

,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate,Private_Yes,Apps
CollegeName,,,,,,,,,,,,,,,,,,
Abilene Christian University,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60,1,1660
Adelphi University,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56,1,2186
Adrian College,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54,1,1428
Agnes Scott College,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59,1,417
Alaska Pacific University,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15,1,193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Worcester State College,1515,543,4,26,3089,2029,6797,3900,500,1200,60,60,21.0,14,4469,40,0,2197
Xavier University,1805,695,24,47,2849,1107,11520,4960,600,1250,73,75,13.3,31,9189,83,1,1959
Xavier University of Louisiana,1915,695,34,61,2793,166,6900,4200,617,781,67,75,14.4,20,8323,49,1,2097


In [13]:
df1.corr()

,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate,Private_Yes,Apps
Accept,1.000000,0.911637,0.192447,0.247476,0.874223,0.441271,-0.025755,0.090899,0.113525,0.200989,0.355758,0.337583,0.176229,-0.159990,0.124717,0.067313,-0.475252,0.943451
Enroll,0.911637,1.000000,0.181294,0.226745,0.964640,0.513069,-0.155477,-0.040232,0.112711,0.280929,0.331469,0.308274,0.237271,-0.180794,0.064169,-0.022341,-0.567908,0.846822
Top10perc,0.192447,0.181294,1.000000,0.891995,0.141289,-0.105356,0.562331,0.371480,0.118858,-0.093316,0.531828,0.491135,-0.384875,0.455485,0.660913,0.494989,0.164132,0.338834
Top25perc,0.247476,0.226745,0.891995,1.000000,0.199445,-0.053577,0.489394,0.331490,0.115527,-0.080810,0.545862,0.524749,-0.294629,0.417864,0.527447,0.477281,0.095752,0.351640
F.Undergrad,0.874223,0.964640,0.141289,0.199445,1.000000,0.570512,-0.215742,-0.068890,0.115550,0.317200,0.318337,0.300019,0.279703,-0.229462,0.018652,-0.078773,-0.615561,0.814491
P.Undergrad,0.441271,0.513069,-0.105356,-0.053577,0.570512,1.000000,-0.253512,-0.061326,0.081200,0.319882,0.149114,0.141904,0.232531,-0.280792,-0.083568,-0.257001,-0.452088,0.398264
Outstate,-0.025755,-0.155477,0.562331,0.489394,-0.215742,-0.253512,1.000000,0.654256,0.038855,-0.299087,0.382982,0.407983,-0.554821,0.566262,0.672779,0.571290,0.552650,0.050159
Room.Board,0.090899,-0.040232,0.371480,0.331490,-0.068890,-0.061326,0.654256,1.000000,0.127963,-0.199428,0.329202,0.374540,-0.362628,0.272363,0.501739,0.424942,0.340532,0.164939
Books,0.113525,0.112711,0.118858,0.115527,0.115550,0.081200,0.038855,0.127963,1.000000,0.179295,0.026906,0.099955,-0.031929,-0.040208,0.112409,0.001061,-0.018549,0.132559
Personal,0.200989,0.280929,-0.093316,-0.080810,0.317200,0.319882,-0.299087,-0.199428,0.179295,1.000000,-0.010936,-0.030613,0.136345,-0.285968,-0.097892,-0.269344,-0.304485,0.178731


We create a correlation matrix to see if there exists any multicollinearity. We see that there is a high correlation between `Accept` and `Enroll` which in practice we should drop one variable, but according to question we need to run the model using all other variables.

In [14]:
train, test = train_test_split(df1, test_size = 0.3) #train test with a 70% to 30% ratio
X_train = train.iloc[0:,0:17]
y_train = train['Apps']
X_test = test.iloc[0:,0:17]
y_test = test['Apps']
X_train

,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate,Private_Yes
CollegeName,,,,,,,,,,,,,,,,,
Prairie View A. and M. University,2234,1061,10,22,4564,448,4290,3500,598,1582,55,93,19.4,1,5967,35,0
Trinity University,1818,601,62,93,2110,95,12240,5150,500,490,94,96,9.6,20,14703,93,1
Trinity College CT,1798,478,46,84,1737,244,18810,5690,500,680,91,96,10.4,48,18034,91,1
Wake Forest University,2392,903,75,88,3499,172,13850,4360,500,1250,95,97,4.3,37,41766,89,1
Seton Hall University,3565,1000,16,36,4384,1530,12000,6484,650,1000,81,84,14.4,15,10080,64,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SUNY at Buffalo,9649,3087,36,100,13963,3124,6550,4731,708,957,90,97,13.6,15,11177,56,0
Eastern Connecticut State University,1493,564,14,50,2766,1531,5962,4316,650,500,71,76,16.9,14,5719,50,0
Immaculata College,253,103,16,44,494,1305,10000,5364,500,1000,56,64,11.2,33,7305,69,1


## b)

In [15]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

test_error = mean_squared_error(y_test, y_pred)
test_error

1203743.8910864787

After fitting a linear regression using the scikitlearn package, we see that the test error generated is `1203743.8910864787`.

## c) 

In [16]:
alpha_list = np.logspace(-20, 20, 10000) #create the list of alphas we want to cross validate
model_Ridge = RidgeCV(alphas = alpha_list)
model_Ridge.fit(X_train, y_train)
y_ridgepred = model_Ridge.predict(X_test)

ridge_test_error = mean_squared_error(y_test, y_ridgepred)
ridge_test_error

1204740.4020278968

After fitting a ridge linear regression using the scikitlearn package and cross validating the values of alpha, we see that the test error generated is `1204740.4020278968`.

## d) 

In [17]:
alpha_list = np.logspace(-20, 20, 10000) #create the list of alphas we want to cross validate
model_Lasso = LassoCV(alphas = alpha_list)
model_Lasso.fit(X_train, y_train)
y_lassopred = model_Lasso.predict(X_test)

lasso_test_error = mean_squared_error(y_test, y_lassopred)
lasso_test_error

1203503.8700627144

After fitting a lasso linear regression using the scikitlearn package and cross validating the values of alpha, we see that the test error generated is `1203503.8700627144`.

In [18]:
list_name = list(df1)
list_name.pop()
list_coef = list(model_Lasso.fit(X_train, y_train).coef_)
list_icp = model_Lasso.fit(X_train, y_train).intercept_

list_name.insert(0, 'Intercept')
list_coef.insert(0, list_icp)

df_coef = df = pd.DataFrame({'Variable': list_name, 'Coefficient': list_coef})
df_coef


,Variable,Coefficient
0,Intercept,-1.316546
1,Accept,1.686477
2,Enroll,-1.045237
3,Top10perc,59.512495
4,Top25perc,-21.955398
5,F.Undergrad,0.061058
6,P.Undergrad,0.066237
7,Outstate,-0.086717
8,Room.Board,0.086307
9,Books,0.047809


Here are the parameter estimates for the lasso regression model. 

## g)

In [19]:
lr_acc = r2_score(y_test, y_pred)
ridge_acc = r2_score(y_test, y_ridgepred)
lasso_acc = r2_score(y_test, y_lassopred)

print('Accuracy score for linear regression is: ' + str(lr_acc))
print('Accuracy score for ridge regression is: ' + str(ridge_acc))
print('Accuracy score for lasso regression is: ' + str(lasso_acc))

Accuracy score for linear regression is: 0.888163962060382
Accuracy score for ridge regression is: 0.8880713793803975
Accuracy score for lasso regression is: 0.8881862616546052


We can see that using the R squared as a accuracy metric for the three models, each of them achieve an accuracy score around 0.89, which indicates that the prediction is pretty accurate. Here is a summary of results obtained: least squares regression test error: `1203743.8910864787`; ridge regression test error: `1204740.4020278968`; lasso regression test error: `1203503.8700627144`. We see that there is not much difference between the test errors of the three approaches. 